In [3]:
# 2017-07-05 
"""
   training a model
"""
from sklearn.cross_validation import train_test_split

from sklearn.externals import joblib
from sklearn import linear_model
import pandas as pd #
import numpy as np
import os
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import r2_score, mean_squared_error

import os
savedir = os.getcwd()

filename1 = os.path.join(savedir, 'static', 'model_columns.pkl')
filename2 = os.path.join(savedir, 'static', 'model.pkl')
filename3 = os.path.join(savedir, 'static', 'housing.csv')
filename4 = os.path.join(savedir, 'static', 'lm_model.pkl')
filename_t = os.path.join(savedir, 'static', 't.pkl')
filename_MSE = os.path.join(savedir, 'static', 'mse.pkl')
filename_XTX_inv = os.path.join(savedir, 'static', 'xtx_inv.pkl')

# Load the Boston housing dataset
boston = pd.read_csv(filename3)
boston = pd.DataFrame(boston)
include = ['crime_rate', 'avg_number_of_rooms', 'distance_to_employment_centers', 'property_tax_rate', 'pupil_teacher_ratio']
features = boston[include].values
#features.columns = include
y = boston['house_value'].values


X_train, X_test, y_train, y_test = train_test_split(features, y, test_size = 0.33, random_state = 1978)


# linear regression

normalize = False
n_jobs = 1
LR = linear_model.LinearRegression(normalize=normalize, n_jobs= n_jobs)
t_value = 2.13144955 # t value to get the 95% conf. interval.
        
x = pd.DataFrame(features)
y = pd.DataFrame(y)
        
LR.fit(x, y)
features_fit = LR.predict(x)
MSE = np.power(y.subtract(features_fit), 2).sum(axis=0) / (x.shape[0] - x.shape[1] - 1)
x.loc[:, 'const_one'] = 1
XTX_inv = np.linalg.inv(np.dot(np.transpose(x.values) , x.values))

model = LR

#joblib.dump(model, filename4)
#joblib.dump(t_value, filename_t)
#joblib.dump(MSE, filename_MSE)
#joblib.dump(XTX_inv, filename_XTX_inv)

# calculate confidence intervals for new data
mean_x = np.mean(x)         # mean of x
n = len(x)              # number of samples in origional fit
t = 2.132   # appropriate t value (where n=5, two tailed 95%)
y_err = y - features_fit      # residuals
s_err = np.sum(np.power(y_err,2))   # sum of the squares of the residuals

a = x.values[[7]]
print(a)
#b = a.reshape(-1,len(a))
#print(b)

#p_x = a
#confs = t * np.sqrt((s_err/(n-2))*(1.0/n + (np.power((p_x-mean_x),2)/((np.sum(np.power(x,2)))-n*(np.power(mean_x,2))))))
 
# now predict y based on test x-values
#p_y = z[0]*p_x+z[0]
 
# get lower and upper confidence limits based on predicted y and confidence intervals
#lower = p_y - abs(confs)
#upper = p_y + abs(confs)



[[  2.11240000e-01   5.63100000e+00   6.08210000e+00   3.11000000e+02
    1.52000000e+01   1.00000000e+00]]


In [5]:
## Random Forest


rfModel = RandomForestRegressor(n_estimators=100, max_features = 2, min_samples_leaf=1)
rfModel.fit(features, y)
# save the model to disk

joblib.dump(rfModel, filename2)

rf = RandomForestRegressor(n_estimators=100, max_features = 2, min_samples_leaf=1)
rf.fit(X_train, y_train)

print("RF test set MSE: %.2f"
      % np.mean((y_test - rf.predict(X_test))**2)) # MSE

print("RF test set R2: %.2f"
      % r2_score(y_test, rf.predict(X_test))) # R2

def pred_ints(model, X, percentile=95):
    err_down = []
    err_up = []
    preds = []
    for pred in model.estimators_:
        preds.append(pred.predict(X)[0])
    err_down.append(np.percentile(preds, (100 - percentile) / 2. ))
    err_up.append(np.percentile(preds, 100 - (100 - percentile) / 2.))
    return err_down, err_up

a = features[1,]
b = a.reshape(-1,len(a))


err_down, err_up = pred_ints(rfModel, b)
pred = rfModel.predict(b)
 
print(err_down, pred, err_up)

/Users/myliheik/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RF test set MSE: 21.95
RF test set R2: 0.76
[32.4375] [ 34.826] [37.614999999999995]
